# A Dialogue-Based SNOMED Prediction Benchmark

# Prepare Data
For data, I decided to use [Synthea](https://synthea.mitre.org/downloads) 1K sample dataset. 

In [3]:
data_root = '../../data/synthea_sample_data_csv_nov2021/'

In [4]:
import pandas as pd
import os

# Load CSVs
patients_df = pd.read_csv(os.path.join(data_root, "patients.csv"))
conditions_df = pd.read_csv(os.path.join(data_root, "conditions.csv"))
observations_df = pd.read_csv(os.path.join(data_root, "observations.csv"))

# Number of patients to sample
SAMPLE_SIZE = 500

# Sample patients
sample_patient_ids = patients_df['Id'].sample(SAMPLE_SIZE, random_state=42).tolist()
sample_patients = patients_df[patients_df['Id'].isin(sample_patient_ids)]
sample_conditions = conditions_df[conditions_df['PATIENT'].isin(sample_patient_ids)]
sample_observations = observations_df[observations_df['PATIENT'].isin(sample_patient_ids)]

# Build structured context per patient
flat_records = []

for _, patient in sample_patients.iterrows():
    pid = patient['Id']
    age = 2025 - int(patient['BIRTHDATE'][:4])
    gender = patient['GENDER']
    city = patient['CITY']
    state = patient['STATE']

    # Conditions with filtering
    raw_conds = (
        sample_conditions[sample_conditions['PATIENT'] == pid]['DESCRIPTION']
        .dropna()
        .unique()
        .tolist()
    )

    # Filter valid condition rows (as DataFrame)
    irrelevant_keywords = ["employment", "education", "finding", "social", "victim", "student", "contact"]
    valid_cond_df = sample_conditions[
        (sample_conditions['PATIENT'] == pid) &
        (~sample_conditions['DESCRIPTION'].str.lower().str.contains('|'.join(irrelevant_keywords)))
    ][['DESCRIPTION', 'CODE']].dropna().drop_duplicates()

    # SNOMED gold codes
    snomed_gold = [
        f"{row['CODE']} - {row['DESCRIPTION']}" for _, row in valid_cond_df.iterrows()
    ]

    # Conditions (for oracle_context)
    filtered_conds = valid_cond_df['DESCRIPTION'].tolist()

    # Observations
    obs_df = sample_observations[sample_observations['PATIENT'] == pid]
    obs_pairs = []
    for _, row in obs_df.iterrows():
        label = row['DESCRIPTION']
        value = row['VALUE']
        if pd.notna(label) and pd.notna(value):
            obs_pairs.append(f"{label}: {value}")

    # Create flat context
    context = f"Age: {age}\nGender: {gender}\nLocation: {city}, {state}"
    if filtered_conds:
        context += "\nConditions: " + "; ".join(filtered_conds[:10])
    if obs_pairs:
        context += "\nObservations:\n- " + "\n- ".join(obs_pairs[:10])

    flat_records.append({
        "patient_id": pid,
        "oracle_context": context,
        "snomed_gold": snomed_gold[:10]
    })

# Optional: Save to JSON
import json
with open("synthea_oracle_context.json", "w") as f:
    json.dump(flat_records, f, indent=2)

# Print a few samples
for record in flat_records[:3]:
    print(json.dumps(record, indent=2))


{
  "patient_id": "339144f8-50e1-633e-a013-f361391c4cff",
  "oracle_context": "Age: 27\nGender: M\nLocation: Boston, Massachusetts\nConditions: Sprain of ankle; Hypertension; Acute viral pharyngitis (disorder); Viral sinusitis (disorder); Refugee (person)\nObservations:\n- Body Height: 166.8\n- Pain severity - 0-10 verbal numeric rating [Score] - Reported: 3.0\n- Body Weight: 70.9\n- Body Mass Index: 25.5\n- Body mass index (BMI) [Percentile] Per age and gender: 94.0\n- Diastolic Blood Pressure: 82.0\n- Systolic Blood Pressure: 129.0\n- Heart rate: 87.0\n- Respiratory rate: 15.0\n- Tobacco smoking status NHIS: Never smoker",
  "snomed_gold": [
    "44465007 - Sprain of ankle",
    "59621000 - Hypertension",
    "195662009 - Acute viral pharyngitis (disorder)",
    "444814009 - Viral sinusitis (disorder)",
    "446654005 - Refugee (person)"
  ]
}
{
  "patient_id": "d488232e-bf14-4bed-08c0-a82f34b6a197",
  "oracle_context": "Age: 22\nGender: F\nLocation: Hingham, Massachusetts\nCondition

In [ ]:
import openai
import asyncio
import json
from tqdm import tqdm

client = openai.AsyncOpenAI(api_key="sk-proj-") # Replace with your actual OpenAI API key to run this script

# Load structured patients
with open("synthea_oracle_context.json", "r") as f:
    patients = json.load(f)

output_file = "oracle_qa_finetune_data.jsonl"

def build_chat_messages(context, snomed_list):
    gold = "\n".join(snomed_list)
    return [
        {
            "role": "system",
            "content": (
                "You are simulating a patient with the following medical conditions. "
                "Respond in plain language without naming diagnoses unless asked. Vary your answers. "
                "Focus on realistic symptoms, complaints, or feelings."
            )
        },
        {
            "role": "user",
            "content": f"""PATIENT INFO:
            {context}

            GOLD CONDITIONS (SNOMED):
            {gold}

            Generate 5 different question/answer pairs that reflect the patient's symptoms or experiences. 
            Do NOT use clinical language. Each answer should sound like something a regular person would say. 
            Only output a JSON array like this:
            [
                {{ "question": "...", "answer": "..." }},
            ...
            ]
            """
        }
    ]

def build_negative_qa_prompt(context, snomed_list):
    gold = "\n".join(snomed_list)
    return [
        {
            "role": "system",
            "content": "You are simulating a patient with limited medical knowledge."
        },
        {
            "role": "user",
            "content": f"""Patient Info:
            {context}

            GOLD CONDITIONS (SNOMED):
            {gold}

            TASK:
            - Imagine a doctor is asking the patient about symptoms NOT related to the conditions above.
            - Generate 2 question/answer pairs.
            - The **question** should come from the doctor, phrased in plain language.
            - The **answer** should sound like the patient is unsure or hasn’t experienced it.
            - The patient should **not explain or speculate** — just admit uncertainty or lack of awareness.

            Only output a JSON array like this:
            [
            {{ "question": "Have you noticed any changes in your memory?", "answer": "I’m not really sure. I haven’t paid much attention to that." }},
            ...
            ]
            """
        }
    ]


# Generate QA examples
# Async function to process one patient
async def process_patient(patient, sem, out_fp):
    async with sem:
        try:
            # GPT call: symptom-based QAs
            res1 = await client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=build_chat_messages(patient["oracle_context"], patient["snomed_gold"]),
                temperature=0.9,
                timeout=20,
            )
            qa_pairs = json.loads(res1.choices[0].message.content.strip())

            # GPT call: negative QAs
            res2 = await client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=build_negative_qa_prompt(patient["oracle_context"], patient["snomed_gold"]),
                temperature=0.8,
                timeout=20,
            )
            neg_pairs = json.loads(res2.choices[0].message.content.strip())
            qa_pairs.extend(neg_pairs)

            # Write to file (append mode)
            for qa in qa_pairs:
                record = {
                    "instruction": "Answer the user's question based on the patient context. Use lay language only.",
                    "snomed_gold": patient["snomed_gold"],
                    "input": f"Patient Info:\n{patient['oracle_context']}\nQuestion: {qa['question']}",
                    "output": qa["answer"]
                }
                out_fp.write(json.dumps(record) + "\n")

        except Exception as e:
            print(f"⚠️ Error for {patient['patient_id']}: {str(e)}")

# Main async runner
async def main():
    output_path = "oracle_qa_finetune_data.json"
    sem = asyncio.Semaphore(10)  # Max 10 concurrent GPT calls

    with open(output_path, "w") as out_fp:
        tasks = [process_patient(p, sem, out_fp) for p in patients]
        for f in tqdm(asyncio.as_completed(tasks), total=len(tasks)):
            await f
 
 # Run the event loop
import nest_asyncio
nest_asyncio.apply()
await main()


  2%|▏         | 11/500 [00:05<03:17,  2.48it/s]

⚠️ Error for 4c7bbf23-68f4-86e8-c61e-1a1b1e81c4ea: Expecting value: line 1 column 1 (char 0)


  7%|▋         | 34/500 [00:10<01:45,  4.43it/s]

⚠️ Error for eb2f8ba1-fd23-fdbb-dbe3-484a3d58244e: Expecting value: line 1 column 1 (char 0)


  8%|▊         | 42/500 [00:13<02:31,  3.02it/s]

...

 97%|█████████▋| 485/500 [02:24<00:03,  4.17it/s]

⚠️ Error for 767e1d64-d100-a9ec-fe05-073c6857ffa8: Expecting value: line 1 column 1 (char 0)


 98%|█████████▊| 488/500 [02:25<00:03,  3.56it/s]

⚠️ Error for 16374251-beab-44f9-ac53-faece4e66bc2: Expecting value: line 1 column 1 (char 0)


100%|██████████| 500/500 [02:29<00:00,  3.34it/s]


Let's try without fine-tune.

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "biomistral/biomistral-7b"
local_dir = "./biomistral-7b"

# Download and cache model/tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=local_dir)
model = AutoModelForCausalLM.from_pretrained(model_id, cache_dir=local_dir)


Test zero-shot!

In [ ]:
import json
import random

# run 5 sample
for _ in range(5):
    # Load a QA sample from your dataset
    with open("oracle_qa_finetune_data.json", "r") as f:
        lines = f.readlines()
        sample = json.loads(random.choice(lines))

    # Unpack the fields
    instruction = sample["instruction"]
    input_block = sample["input"]
    true_output = sample["output"]


    # Format prompt using your real training example
    prompt = f"""### Instruction:
    You are a patient. Answer the doctor's question based on the patient context given. 
    Use lay language only. Please be concise and answer in one or two short sentences.

    ### Input:
    {input_block}

    ### Response:
    """

    # Tokenize and generate
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,
            do_sample=False,
            temperature=0.6
        )

    # Decode
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    model_response = response.split("### Response:")[-1].strip()

    # Show comparison
    print("\n🔍 Input Prompt:\n", prompt)
    print("\n🧠 Model Response:\n", model_response)
    print("\n✅ Expected (Fine-Tuned) Answer:\n", true_output)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
c:\Users\young\anaconda3\envs\aihc-ml\lib\site-packages\transformers\models\mistral\modeling_mistral.py:456: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



🔍 Input Prompt:
 ### Instruction:
    You are a patient. Answer the doctor's question based on the patient context given. 
    Use lay language only. Please be concise and answer in one or two short sentences.

    ### Input:
    Patient Info:
Age: 114
Gender: F
Location: Westfield, Massachusetts
Conditions: Refugee (person); Prediabetes; Anemia (disorder); Miscarriage in first trimester; Tubal pregnancy; Rheumatoid arthritis; Chronic sinusitis (disorder); Polyp of colon; Osteoporosis (disorder); Chronic intractable migraine without aura
Observations:
- Body Height: 161.9
- Pain severity - 0-10 verbal numeric rating [Score] - Reported: 2.0
- Body Weight: 72.9
- Body Mass Index: 27.8
- Diastolic Blood Pressure: 80.0
- Systolic Blood Pressure: 112.0
- Heart rate: 75.0
- Respiratory rate: 16.0
- Glucose: 65.4
- Urea Nitrogen: 19.6
Question: How have your sinus issues been lately?

    ### Response:
    

🧠 Model Response:
 1. My sinus issues have been pretty bad lately. I've been having 

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



🔍 Input Prompt:
 ### Instruction:
    You are a patient. Answer the doctor's question based on the patient context given. 
    Use lay language only. Please be concise and answer in one or two short sentences.

    ### Input:
    Patient Info:
Age: 65
Gender: M
Location: North Eastham, Massachusetts
Conditions: Hypertension; Cardiac Arrest; History of cardiac arrest (situation); Viral sinusitis (disorder); Laceration of foot; Otitis media
Observations:
- Body Height: 186.8
- Pain severity - 0-10 verbal numeric rating [Score] - Reported: 2.0
- Body Weight: 85.2
- Body Mass Index: 24.4
- Diastolic Blood Pressure: 83.0
- Systolic Blood Pressure: 115.0
- Heart rate: 74.0
- Respiratory rate: 16.0
- Tobacco smoking status NHIS: Never smoker
- In the past year  have you been afraid of your partner or ex-partner?: No
Question: Do you have any trouble with blurry vision or seeing double?

    ### Response:
    

🧠 Model Response:
 1. No, I do not have any trouble with blurry vision or seeing d

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



🔍 Input Prompt:
 ### Instruction:
    You are a patient. Answer the doctor's question based on the patient context given. 
    Use lay language only. Please be concise and answer in one or two short sentences.

    ### Input:
    Patient Info:
Age: 13
Gender: F
Location: East Brookfield, Massachusetts
Conditions: Acute bronchitis (disorder); Streptococcal sore throat (disorder); Sinusitis (disorder); Chronic sinusitis (disorder)
Observations:
- Body Height: 51.7
- Pain severity - 0-10 verbal numeric rating [Score] - Reported: 2.0
- Body Weight: 3.7
- Weight-for-length Per age and sex: 24.0
- Head Occipital-frontal circumference: 34.8
- Diastolic Blood Pressure: 85.0
- Systolic Blood Pressure: 109.0
- Heart rate: 72.0
- Respiratory rate: 15.0
- Leukocytes [#/volume] in Blood by Automated count: 10.0
Question: How have you been feeling lately?

    ### Response:
    

🧠 Model Response:
 ive been feeling really sick and tired.

✅ Expected (Fine-Tuned) Answer:
 I've been feeling really co

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



🔍 Input Prompt:
 ### Instruction:
    You are a patient. Answer the doctor's question based on the patient context given. 
    Use lay language only. Please be concise and answer in one or two short sentences.

    ### Input:
    Patient Info:
Age: 7
Gender: F
Location: Somerville, Massachusetts
Observations:
- Body Height: 49.4
- Pain severity - 0-10 verbal numeric rating [Score] - Reported: 2.0
- Body Weight: 3.1
- Weight-for-length Per age and sex: 32.4
- Head Occipital-frontal circumference: 33.2
- Diastolic Blood Pressure: 82.0
- Systolic Blood Pressure: 117.0
- Heart rate: 67.0
- Respiratory rate: 14.0
- Leukocytes [#/volume] in Blood by Automated count: 8.9
Question: Are you feeling tired or energetic lately?

    ### Response:
    

🧠 Model Response:
 7-year-old girl feeling energetic lately.

✅ Expected (Fine-Tuned) Answer:
 I've been feeling pretty energetic lately!

🔍 Input Prompt:
 ### Instruction:
    You are a patient. Answer the doctor's question based on the patient co

Fine tune!

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from datasets import load_dataset
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
from transformers import Trainer, DataCollatorForLanguageModeling

model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    torch_dtype=torch.float16,
    device_map=None  # ✅ Do NOT move to device yet
)

tokenizer = AutoTokenizer.from_pretrained(model_dir)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Apply LoRA
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)
model = get_peft_model(model, peft_config)
model = model.to("cuda")

# Load your dataset
data = load_dataset("json", data_files="oracle_qa_finetune_data.jsonl", split="train")

# Format prompt
def format_example(example):
    prompt = f"""### Instruction:
You are a patient. Answer the doctor's question based on the patient context given. 
Use lay language only. Please be concise and answer in one or two short sentences.

### Input:
{example['input']}

### Response:
{example['output']}"""

    return {"text": prompt}

data = data.map(format_example)
tokenized = data.map(lambda x: tokenizer(x["text"], truncation=True, padding="max_length", max_length=512), batched=True)

# Training config
training_args = TrainingArguments(
    output_dir="./biomistral_oracle_lora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-5,
    fp16=True,
    logging_steps=10,
    save_steps=200,
    save_total_limit=2,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

trainer.train()

model.save_pretrained("biomistral_oracle_lora")
tokenizer.save_pretrained("biomistral_oracle_lora")

  0%|          | 0/2382 [00:00<?, ?it/s]

{'loss': 2.0434, 'grad_norm': 10.874444961547852, 'learning_rate': 1.9916036943744756e-05, 'epoch': 0.01}
{'loss': 1.7315, 'grad_norm': 12.161198616027832, 'learning_rate': 1.9832073887489506e-05, 'epoch': 0.03}
{'loss': 1.4156, 'grad_norm': 11.05747127532959, 'learning_rate': 1.974811083123426e-05, 'epoch': 0.04}
{'loss': 1.0928, 'grad_norm': 11.315916061401367, 'learning_rate': 1.966414777497901e-05, 'epoch': 0.05}
{'loss': 0.8984, 'grad_norm': 11.166594505310059, 'learning_rate': 1.9580184718723762e-05, 'epoch': 0.06}
{'loss': 0.7656, 'grad_norm': 8.333418846130371, 'learning_rate': 1.9496221662468516e-05, 'epoch': 0.08}
{'loss': 0.6807, 'grad_norm': 9.426526069641113, 'learning_rate': 1.9412258606213266e-05, 'epoch': 0.09}
{'loss': 0.6128, 'grad_norm': 8.430987358093262, 'learning_rate': 1.932829554995802e-05, 'epoch': 0.1}
{'loss': 0.6199, 'grad_norm': 11.042398452758789, 'learning_rate': 1.9244332493702774e-05, 'epoch': 0.11}
{'loss': 0.553, 'grad_norm': 13.021499633789062, 'lear

('biomistral_oracle_lora\\tokenizer_config.json',
 'biomistral_oracle_lora\\special_tokens_map.json',
 'biomistral_oracle_lora\\tokenizer.json')

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# Path to LoRA adapter and tokenizer
adapter_path = "biomistral_oracle_lora"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(adapter_path)

# Load base model (BioMistral-7B) — assumes you have it locally or access to Hugging Face
base_model_dir = "./biomistral-7b/models--biomistral--biomistral-7b/snapshots/9a11e1ffa817c211cbb52ee1fb312dc6b61b40a5"
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_dir,
    device_map="auto",
    torch_dtype="auto"
)

# Load LoRA adapter
model = PeftModel.from_pretrained(
    base_model,
    adapter_path,
    torch_dtype="auto"
)

model.eval()  # Optional: set to eval mode for inference

Some parameters are on the meta device because they were offloaded to the cpu.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): 

In [9]:
model = model.to("cuda")
next(model.parameters()).device


device(type='cuda', index=0)

Let's evaluate with some random samples.

In [10]:
import json
import random

# run 5 sample
for _ in range(5):
    # Load a QA sample from your dataset
    with open("oracle_qa_finetune_data.json", "r") as f:
        lines = f.readlines()
        sample = json.loads(random.choice(lines))

    # Unpack the fields
    instruction = sample["instruction"]
    input_block = sample["input"]
    true_output = sample["output"]


    # Format prompt using your real training example
    prompt = f"""### Instruction:
    You are a patient. Answer the doctor's question based on the patient context given. 
    Use lay language only. Please be concise and answer in one or two short sentences.

    ### Input:
    {input_block}

    ### Response:
    """

    # Tokenize and generate
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,
            do_sample=False,
            temperature=0.6
        )

    # Decode
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    model_response = response.split("### Response:")[-1].strip()

    # Show comparison
    print("\n🔍 Input Prompt:\n", prompt)
    print("\n🧠 Model Response:\n", model_response)
    print("\n✅ Expected (Fine-Tuned) Answer:\n", true_output)

c:\Users\young\anaconda3\envs\aihc-ml\lib\site-packages\transformers\generation\configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
c:\Users\young\anaconda3\envs\aihc-ml\lib\site-packages\transformers\models\mistral\modeling_mistral.py:456: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



🔍 Input Prompt:
 ### Instruction:
    You are a patient. Answer the doctor's question based on the patient context given. 
    Use lay language only. Please be concise and answer in one or two short sentences.

    ### Input:
    Patient Info:
Age: 30
Gender: M
Location: Boston, Massachusetts
Conditions: Perennial allergic rhinitis with seasonal variation; Chronic sinusitis (disorder); Acute bronchitis (disorder); Viral sinusitis (disorder); Laceration of foot; Streptococcal sore throat (disorder); Suspected COVID-19; COVID-19
Observations:
- Peanut IgE Ab in Serum: 0.0
- Walnut IgE Ab in Serum: 0.3
- Codfish IgE Ab in Serum: 0.3
- Shrimp IgE Ab in Serum: 0.3
- Wheat IgE Ab in Serum: 0.0
- Egg white IgE Ab in Serum: 0.3
- Soybean IgE Ab in Serum: 0.2
- Cow milk IgE Ab in Serum: 0.3
- White oak IgE Ab in Serum: 0.1
- Common Ragweed IgE Ab in Serum: 0.1
Question: How is your foot feeling after the laceration?

    ### Response:
    

🧠 Model Response:
 It's been healing well, but it's s

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



🔍 Input Prompt:
 ### Instruction:
    You are a patient. Answer the doctor's question based on the patient context given. 
    Use lay language only. Please be concise and answer in one or two short sentences.

    ### Input:
    Patient Info:
Age: 73
Gender: F
Location: Northampton, Massachusetts
Conditions: Miscarriage in first trimester; Normal pregnancy; Viral sinusitis (disorder); Acute bronchitis (disorder); Primary fibromyalgia syndrome; Chronic obstructive bronchitis (disorder); Osteoarthritis of knee
Observations:
- Body Height: 164.3
- Pain severity - 0-10 verbal numeric rating [Score] - Reported: 2.0
- Body Weight: 75.4
- Body Mass Index: 27.9
- Diastolic Blood Pressure: 86.0
- Systolic Blood Pressure: 123.0
- Heart rate: 67.0
- Respiratory rate: 14.0
- Total Cholesterol: 185.3
- Triglycerides: 146.2
Question: Have you experienced any sudden vision changes?

    ### Response:
    

🧠 Model Response:
 I'm not sure. I haven't really paid attention to my vision.

✅ Expected (F

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



🔍 Input Prompt:
 ### Instruction:
    You are a patient. Answer the doctor's question based on the patient context given. 
    Use lay language only. Please be concise and answer in one or two short sentences.

    ### Input:
    Patient Info:
Age: 67
Gender: M
Location: Pepperell, Massachusetts
Conditions: Hypertension; Chronic sinusitis (disorder); Viral sinusitis (disorder); Coronary Heart Disease; Acute viral pharyngitis (disorder); Stroke
Observations:
- Body Height: 174.5
- Pain severity - 0-10 verbal numeric rating [Score] - Reported: 3.0
- Body Weight: 87.3
- Body Mass Index: 28.7
- Diastolic Blood Pressure: 108.0
- Systolic Blood Pressure: 170.0
- Heart rate: 98.0
- Respiratory rate: 12.0
- Tobacco smoking status NHIS: Never smoker
- In the past year  have you been afraid of your partner or ex-partner?: No
Question: Have you experienced any unusual skin rashes or itching lately?

    ### Response:
    

🧠 Model Response:
 I'm not sure. I haven't really noticed anything like t

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



🔍 Input Prompt:
 ### Instruction:
    You are a patient. Answer the doctor's question based on the patient context given. 
    Use lay language only. Please be concise and answer in one or two short sentences.

    ### Input:
    Patient Info:
Age: 58
Gender: F
Location: Holyoke, Massachusetts
Conditions: Hyperlipidemia; Viral sinusitis (disorder); Normal pregnancy; Miscarriage in first trimester; Blighted ovum; Localized  primary osteoarthritis of the hand; Fracture subluxation of wrist; Suspected lung cancer (situation); Non-small cell lung cancer (disorder); Anemia (disorder)
Observations:
- Glucose: 92.8
- Urea Nitrogen: 8.5
- Creatinine: 3.1
- Calcium: 9.3
- Sodium: 136.9
- Potassium: 4.3
- Chloride: 101.8
- Carbon Dioxide: 27.1
- Glomerular filtration rate/1.73 sq M.predicted: 78.3
- Protein [Mass/volume] in Serum or Plasma: 64.8
Question: Do you experience frequent headaches?

    ### Response:
    

🧠 Model Response:
 I'm not sure. I don't think I've had any headaches recently

It seems to behave much better.